In [1]:
# Generate data for problem.
import numpy as np
np.random.seed(1)
m = 25
n = 10
A = np.random.randn(m, n)
b = np.random.randn(m, 1)

In [2]:
# Form and solve problem.
from cvxpy import *
gamma = Parameter(sign="positive")
gamma.value = 1
x = Variable(n)
cost = sum_squares(A*x-b) + gamma*norm(x,1)
prob = Problem(Minimize(cost),
               [norm(x,"inf") <= 1])
opt_val = prob.solve()
solution = x.value

In [ ]:
# For loop style trade-off curve.
gamma_vals = np.logspace(-4, 2, 100)
x_values = []
for val in gamma_vals:
    gamma.value = val
    prob.solve()
    x_values.append(x.value)

In [ ]:
# Parallel style trade-off curve.

# Use tools for parallelism in standard library.
from multiprocessing import Pool

# Function maps gamma value to optimal x.
def get_x(gamma_value):
    gamma.value = gamma_value
    result = prob.solve()
    return x.value

# Parallel computation with N processes.
N = 4
pool = Pool(processes = N)
x_values = pool.map(get_x, gamma_vals)

In [ ]:
# Plot regularization path.
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

for i in range(n):
    plt.plot(gamma_vals, [xi[i,0] for xi in x_values])
plt.xlabel(r"$\gamma$", fontsize=16)
plt.ylabel(r"$x_i$", fontsize=16)
plt.xscale("log")
plt.title(r"Entries of $x$ versus $\gamma$", fontsize=16)